In [1]:
import util_mnist_reader
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

#Data Preprocessing
#Split Data
X_train, trainY = util_mnist_reader.load_mnist('../data/fashion', kind='train')
X_test, testY = util_mnist_reader.load_mnist('../data/fashion', kind='t10k')

#Normaize the data
X_train = X_train / 255.0
X_test = X_test / 255.0

#Split test data into validation and test
X_val, X_test, valY, testY = train_test_split(X_test, testY, test_size=0.5)


C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passi

In [2]:
#calculate and return sigmoid value
def sigmoid(z):
    return 1/(1 + np.exp(-z))

#calculate and return softmax value
def softmax(z):
    a = np.exp(z - z.max(axis=0, keepdims=True))
    return a / a.sum(axis = 0, keepdims=True)

#calculate and return loss value
def calculate_loss(A2, Y):
    loss = np.mean(-Y * np.log(A2 + 0.0000001))
    return loss

In [3]:
def Task_1(y_train, y_test, y_val):
    #Convert into hot vector
    y_train = tf.keras.utils.to_categorical(y_train, 10)
    y_test = tf.keras.utils.to_categorical(y_test, 10)
    y_val = tf.keras.utils.to_categorical(y_val, 10)
    
    # Set the hyperparameters
    m = 60000
    #No. of neurons in first layer
    n_x = 784    
    #No. of neurons in hidden layer
    n_h = 800    
    #No. of neurons in output layer
    n_y = 10     
    epochs = 60
    learning_rate = 0.03
    training_loss = []
    training_accuracy = []
    validation_loss = []
    validation_accuracy = []
    
    #Initialise weights and bias
    W1 = np.random.randn(n_h, n_x)
    b1 = np.ones((n_h, 1))
    W2 = np.random.randn(n_y, n_h)
    b2 = np.ones((n_y, 1))
    for i in range(0, epochs):
        #Forward Propogation
        Z1 = np.dot(W1, X_train.T) + b1
        A1 = sigmoid(Z1)
        Z2 = np.dot(W2, A1) + b2
        A2 = softmax(Z2)
        
        #calculate training loss
        loss = calculate_loss(A2, y_train.T)
        training_loss.append(loss)        
        
        #calculate training accuracy
        y_pred_labels = np.argmax(A2.T, axis=1)
        accuracy = accuracy_score(trainY, y_pred_labels)
        print("Iteration",i, "Training_Loss  "," --- ",loss,"---","Training_accuracy"," --- ",accuracy)     
        training_accuracy.append(accuracy)
        
        #validate the data
        Z1_v = np.dot(W1, X_val.T) + b1
        A1_v = sigmoid(Z1_v)
        Z2_v = np.dot(W2, A1_v) + b2
        A2_v = softmax(Z2_v)
        
        #calculate validation loss
        val_loss = calculate_loss(A2_v, y_val.T)
        validation_loss.append(val_loss)
        
        #calculate validation accuracy
        y_val_pred_labels = np.argmax(A2_v.T, axis=1)
        val_accuracy = accuracy_score(valY, y_val_pred_labels) 
        print("Iteration",i, "validation_Loss"," --- ",val_loss,"---","Validation_accuracy"," --- ",val_accuracy)
        validation_accuracy.append(val_accuracy)
        
        #Backward Propogation
        dZ2 = A2 - y_train.T
        dW2 = np.dot(dZ2, A1.T)/m
        db2 = np.sum(dZ2, axis=1, keepdims=True)/m
        
        W2 = W2 - learning_rate*dW2
        b2 = b2 - learning_rate*db2
        
        dZ1 = np.multiply(np.dot(W2.T, dZ2), 1-np.power(A1, 2))
        dW1 = np.dot(dZ1, X_train)/m
        db1 = np.sum(dZ1, axis=1, keepdims=True)/m
        
        W1 = W1 - learning_rate*dW1
        b1 = b1 - learning_rate*db1
     
    #test the model using test data, updated bias and weights from trained model
    test_Z1 = np.dot(W1, X_test.T) + b1    
    test_A1 = sigmoid(test_Z1)
    test_Z2 = np.dot(W2, test_A1) + b2
    test_A2 = softmax(test_Z2)
    test_loss = calculate_loss(test_A2, y_test.T)
    
    #calculate accuracy and print confusion matrix
    test = np.argmax(test_A2, axis=0)
    print('\nTest accuracy:', accuracy_score(testY, test))
    print('\nTest Loss:', test_loss)
    print("\n\nConfusion Matrix:")
    print(confusion_matrix(testY, test))
    
    #Plot Loss/Accuracy VS Epochs
    plt.plot(training_loss, label = 'Training_Loss')
    plt.plot(training_accuracy, label = 'Training_Loss')
    plt.plot(validation_loss, label = 'Training_Loss')
    plt.plot(validation_accuracy, label = 'Training_Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epochs')
    plt.title("Loss/Accuracy VS Epochs")
    plt.legend()
    print('\nTask_1 Complete')

In [4]:
def Task_2():
    #Write to logs through tensor callback to plot graphs
    logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

    model = keras.Sequential()
    #Layer1
    model.add(tf.keras.layers.Dense(200, activation='relu', input_shape=(784, )))
    model.add(tf.keras.layers.Dropout(0.2))
    #Layer2
    model.add(tf.keras.layers.Dense(300, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.4))
    #Layer3
    model.add(tf.keras.layers.Dense(50, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.2))
    #Layer4
    model.add(tf.keras.layers.Dense(10, activation='softmax'))

    model.summary()
    
    model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.Adam(lr=0.001), metrics=['accuracy']) #sgd tf.keras.optimizers.Adam(lr=0.003)
    
    model.fit(X_train, trainY, batch_size=500, epochs=5, validation_data=(X_val, valY), callbacks=[tensorboard_callback])

    test_loss, test_acc = model.evaluate(X_test,  testY, verbose=2)

    prediction = model.predict(X_test)
    
    y_pred_labels = np.argmax(prediction,axis=1)
    print("\n\nConfusion Matrix:")
    print(confusion_matrix(testY, y_pred_labels))
    print('\nTest accuracy:', test_acc)
    print('\nTest Loss:', test_loss)
    print('\nTask_2 Complete')

In [5]:
def Task_3(X_train, X_val, X_test):
    X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
    X_val = X_val.reshape(X_val.shape[0], 28, 28, 1)
    X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

    #Write to logs through tensor callback to plot graphs
    logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)
    
    model = tf.keras.Sequential()
    #Convolution Layer1
    model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=2, padding='same', input_shape=(28, 28, 1), activation='relu')) 
    #Max Pooling layer
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    model.add(tf.keras.layers.Dropout(0.2))

    #Convolution Layer2
    model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
    #Max Pooling layer
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    model.add(tf.keras.layers.Dropout(0.2))
    
    #Fully Connected Layer
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    
    model.summary()
    
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    model.fit(X_train, trainY, epochs=5, validation_data=(X_val, valY), callbacks=[tensorboard_callback])

    X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
    test_loss, test_acc = model.evaluate(X_test,  testY, verbose=2)
    
    prediction = model.predict(X_test)
    
    y_pred_labels = np.argmax(prediction,axis=1)
    print("\n\nConfusion Matrix:")
    print(confusion_matrix(testY, y_pred_labels))
    print('\nTest accuracy:', test_acc)
    print('\nTest Loss:', test_loss)
    print('\nTask_3 Complete')

In [ ]:
print("Task_1")
Task_1(trainY, testY, valY)

Task_1
Iteration 0 Training_Loss    ---  1.3657829613657166 --- Training_accuracy  ---  0.061783333333333336
Iteration 0 validation_Loss  ---  1.3675992159861015 --- Validation_accuracy  ---  0.0606
Iteration 1 Training_Loss    ---  1.1440395282461773 --- Training_accuracy  ---  0.0974
Iteration 1 validation_Loss  ---  1.1413452109979418 --- Validation_accuracy  ---  0.1012


In [ ]:
print("Task_2")
Task_2()

In [ ]:
print("Task_3")
Task_3(X_train, X_val, X_test)